In [1]:
from pyspark.sql import SQLContext
import pyspark

sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)

df = sqlContext.read.load('file:///home/hadoop/Downloads/big-data-4/daily_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

23/04/10 02:30:23 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/10 02:30:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 02:30:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/hadoop/anaconda3/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.24357700987383,0,1094
air_pressure_9am,1092,918.8825513138094,3.184161180386833,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287072,11.175514003175877,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.2355107005759,69.13785928889189,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.5528134655317185,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.95351796516923,67.23801294602953,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.598209170780958,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.5939521253574893,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1598.0787786601481,0.0,17704.0


In [3]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



This says that there are 1090 rows. The total number of rows in the DataFrame is 1095:

In [4]:
df.count()

1095

This means that 5 of the rows in the air_temp_9am column are missing values.

In [5]:
removeAllDF = df.na.drop()

In [6]:
removeAllDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.168033449415704|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [7]:
removeAllDF.count()

1064

In [8]:
from pyspark.sql.functions import avg

imputeDF = df

In [12]:
removeAllDF.agg(avg('air_temp_9am')).first(), removeAllDF.agg(avg('air_temp_9am')).first()[0]

(Row(avg(air_temp_9am)=65.02260949558733), 65.02260949558733)

Iterating through each column in the DataFrame: we compute the mean value for that column and then replace any missing values in that column with the mean.

In [13]:
for x in imputeDF.columns:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x,':', meanValue)
    imputeDF = imputeDF.na.fill(meanValue, [x])

number : 545.0018796992481
air_pressure_9am : 918.9031798641051
air_temp_9am : 65.02260949558733
avg_wind_direction_9am : 142.30675564934037
avg_wind_speed_9am : 5.48579305071369
max_wind_direction_9am : 148.48042413321315
max_wind_speed_9am : 6.999713658875691
rain_accumulation_9am : 0.18202347650615522
rain_duration_9am : 266.3936973996037
relative_humidity_9am : 34.07743985327709
relative_humidity_3pm : 35.14838093290533


In [14]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219818|
| stddev| 11.14994819992023|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [15]:
imputeDF.count()

1095

In [20]:
1 and 1, 1 & 1

(1, 1)

If we remove all missing values from the data, how many air pressure at 9am measurements have values between 911.736 and 914.67?

In [35]:
removeAllDF.filter((removeAllDF.air_pressure_9am > 911.736) & (removeAllDF.air_pressure_9am < 914.67)).count()

77

If we impute the missing values with the minimum value, how many air temperature at 9am measurements are less than 42.292?

In [46]:
from pyspark.sql.functions import min

In [50]:
removeAllDF.select(min('air_temp_9am')).show()
removeAllDF.agg(min('air_temp_9am')).first()[0]

+------------------+
| min(air_temp_9am)|
+------------------+
|36.752000000000685|
+------------------+



36.752000000000685

In [52]:
imputeDF_min = df

In [53]:
for x in imputeDF_min.columns:
    minValue = removeAllDF.agg(min(x)).first()[0]
    print(x,':', minValue)
    imputeDF_min = imputeDF_min.na.fill(minValue, [x])

number : 0
air_pressure_9am : 907.9900000000024
air_temp_9am : 36.752000000000685
avg_wind_direction_9am : 15.500000000000046
avg_wind_speed_9am : 0.69345139999974
max_wind_direction_9am : 28.89999999999991
max_wind_speed_9am : 1.1855782000000479
rain_accumulation_9am : 0.0
rain_duration_9am : 0.0
relative_humidity_9am : 6.090000000001012
relative_humidity_3pm : 5.3000000000006855


In [54]:
imputeDF_min.filter((imputeDF_min.air_temp_9am < 42.292)).count()

28